In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Supress auto scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
nyc_top_zipcodes_all_years = pd.read_csv("./zipcode_data/nyc_top_zipcodes_all_years.csv")
display(nyc_top_zipcodes_all_years.head())

,zipcode,borough,year,count
0,10469,BRONX,2022,122
1,10466,BRONX,2022,119
2,10463,BRONX,2018,105
3,10466,BRONX,2021,99
4,10456,BRONX,2022,98


In [4]:
zillow_rental_price = pd.read_csv('../data/clean/zrentals.csv', low_memory=False)
zillow_rental_price["date"] = pd.to_datetime(zillow_rental_price["date"]).apply(lambda x: x.year)
zillow_rental_price = zillow_rental_price.rename(columns={"date": "year", "zip_code": "zipcode"})
display(zillow_rental_price.head())

,zipcode,state,city,borough,year,mean_rent
0,11385,NY,New York,QUEENS,2015,2216.569
1,10467,NY,New York,BRONX,2015,1303.889
2,11226,NY,New York,BROOKLYN,2015,1909.384
3,11220,NY,New York,BROOKLYN,2015,1686.422
4,11207,NY,New York,BROOKLYN,2015,2113.610


In [5]:
zillow_rental_price.groupby("borough")['mean_rent'].describe()

,count,mean,std,min,25%,50%,75%,max
borough,,,,,,,,
BRONX,711.000,1836.949,308.809,1254.235,1610.183,1801.508,2017.836,2691.070
BROOKLYN,2665.000,2514.905,584.081,1281.838,2078.844,2404.182,2933.604,4916.624
MANHATTAN,3847.000,3269.474,935.231,1678.251,2557.405,3232.040,3677.203,7976.155
QUEENS,1360.000,2280.403,420.937,1438.286,2023.218,2202.168,2431.803,4121.018
STATEN_ISLAND,153.000,1873.057,292.675,1134.746,1653.635,1915.653,2050.506,2589.630


In [6]:
min_year, max_year = nyc_top_zipcodes_all_years["year"].min(), nyc_top_zipcodes_all_years["year"].max()
print("MIN={}\nMAX={}".format(min_year, max_year))

MIN=2015
MAX=2022


In [7]:
zillow_rental_price = zillow_rental_price[zillow_rental_price["year"].between(min_year, max_year, inclusive=True)]
display(zillow_rental_price)

/var/folders/2r/0fj15shj34z2mbldl18cp9kc0000gp/T/ipykernel_23499/3280843126.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  zillow_rental_price = zillow_rental_price[zillow_rental_price["year"].between(min_year, max_year, inclusive=True)]


,zipcode,state,city,borough,year,mean_rent
0,11385,NY,New York,QUEENS,2015,2216.569
1,10467,NY,New York,BRONX,2015,1303.889
2,11226,NY,New York,BROOKLYN,2015,1909.384
3,11220,NY,New York,BROOKLYN,2015,1686.422
4,11207,NY,New York,BROOKLYN,2015,2113.610
...,...,...,...,...,...,...
8731,10282,NY,New York,MANHATTAN,2022,7836.528
8732,10069,NY,New York,MANHATTAN,2022,5778.583
8733,10004,NY,New York,MANHATTAN,2022,4823.220
8734,10006,NY,New York,MANHATTAN,2022,4180.541


In [9]:
zillow_rental_price = zillow_rental_price.groupby(["zipcode", "year", "borough"])["mean_rent"].mean().to_frame().reset_index()
display(zillow_rental_price)

,zipcode,year,borough,mean_rent
0,10001,2015,MANHATTAN,4010.002
1,10001,2016,MANHATTAN,4022.986
2,10001,2017,MANHATTAN,3958.379
3,10001,2018,MANHATTAN,4001.064
4,10001,2019,MANHATTAN,4145.263
...,...,...,...,...
783,11435,2019,QUEENS,1708.722
784,11435,2020,QUEENS,1747.118
785,11435,2021,QUEENS,1778.253
786,11435,2022,QUEENS,1904.273


In [10]:
non_intersecting = zillow_rental_price.merge(nyc_top_zipcodes_all_years, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)


/var/folders/2r/0fj15shj34z2mbldl18cp9kc0000gp/T/ipykernel_23499/1049469291.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  non_intersecting = zillow_rental_price.merge(nyc_top_zipcodes_all_years, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)


In [21]:
non_intersecting

,zipcode,year,borough,mean_rent,count
0,10001,2015,MANHATTAN,4010.002,NaN
1,10001,2016,MANHATTAN,4022.986,NaN
2,10001,2017,MANHATTAN,3958.379,NaN
3,10001,2018,MANHATTAN,4001.064,NaN
4,10001,2019,MANHATTAN,4145.263,NaN
...,...,...,...,...,...
783,11435,2019,QUEENS,1708.722,NaN
784,11435,2020,QUEENS,1747.118,NaN
785,11435,2021,QUEENS,1778.253,NaN
786,11435,2022,QUEENS,1904.273,NaN


In [14]:
bronx_rent = non_intersecting[non_intersecting["borough"] == "BRONX"]
display(bronx)

,zipcode,year,borough,mean_rent,count
355,10452,2022,BRONX,1926.463,NaN
356,10453,2022,BRONX,1857.340,NaN
357,10454,2022,BRONX,2595.520,NaN
358,10455,2022,BRONX,2090.263,NaN
360,10457,2015,BRONX,1301.131,NaN
...,...,...,...,...,...
416,10467,2022,BRONX,1950.805,NaN
417,10468,2022,BRONX,1796.510,NaN
419,10471,2022,BRONX,2323.055,NaN
420,10472,2022,BRONX,2164.620,NaN


In [16]:
bronx_rent.to_csv("bronx_rent_remaining_zipcodes.csv", index=False)

In [17]:
bronx_rent_csv = pd.read_csv("bronx_rent_remaining_zipcodes.csv")

bronx_rent_csv

,zipcode,year,borough,mean_rent,count
0,10452,2022,BRONX,1926.463,NaN
1,10453,2022,BRONX,1857.340,NaN
2,10454,2022,BRONX,2595.520,NaN
3,10455,2022,BRONX,2090.263,NaN
4,10457,2015,BRONX,1301.131,NaN
...,...,...,...,...,...
58,10467,2022,BRONX,1950.805,NaN
59,10468,2022,BRONX,1796.510,NaN
60,10471,2022,BRONX,2323.055,NaN
61,10472,2022,BRONX,2164.620,NaN


In [22]:
queens_rent = non_intersecting[non_intersecting["borough"] == "QUEENS"]
display(queens_rent)

,zipcode,year,borough,mean_rent,count
422,11101,2015,QUEENS,2753.774,NaN
423,11101,2016,QUEENS,2826.212,NaN
424,11101,2017,QUEENS,2879.503,NaN
425,11101,2018,QUEENS,2873.938,NaN
426,11101,2019,QUEENS,3050.316,NaN
...,...,...,...,...,...
783,11435,2019,QUEENS,1708.722,NaN
784,11435,2020,QUEENS,1747.118,NaN
785,11435,2021,QUEENS,1778.253,NaN
786,11435,2022,QUEENS,1904.273,NaN


In [23]:
queens_rent.to_csv("queens_rent_remaining_zipcodes.csv", index=False)

In [24]:
queens_rent_csv = pd.read_csv("queens_rent_remaining_zipcodes.csv")

queens_rent_csv

,zipcode,year,borough,mean_rent,count
0,11101,2015,QUEENS,2753.774,NaN
1,11101,2016,QUEENS,2826.212,NaN
2,11101,2017,QUEENS,2879.503,NaN
3,11101,2018,QUEENS,2873.938,NaN
4,11101,2019,QUEENS,3050.316,NaN
...,...,...,...,...,...
121,11435,2019,QUEENS,1708.722,NaN
122,11435,2020,QUEENS,1747.118,NaN
123,11435,2021,QUEENS,1778.253,NaN
124,11435,2022,QUEENS,1904.273,NaN


In [26]:
brooklyn_rent = non_intersecting[non_intersecting["borough"] == "BROOKLYN"]
display(brooklyn_rent)

,zipcode,year,borough,mean_rent,count
473,11201,2015,BROOKLYN,3227.864,NaN
474,11201,2016,BROOKLYN,3271.231,NaN
475,11201,2017,BROOKLYN,3292.957,NaN
476,11201,2018,BROOKLYN,3352.501,NaN
477,11201,2019,BROOKLYN,3488.163,NaN
...,...,...,...,...,...
704,11249,2018,BROOKLYN,3566.972,NaN
705,11249,2019,BROOKLYN,3718.571,NaN
706,11249,2020,BROOKLYN,3812.068,NaN
707,11249,2021,BROOKLYN,3999.540,NaN


In [28]:
brooklyn_rent.to_csv("brooklyn_rent_remaining_zipcodes.csv", index=False)

In [29]:
brooklyn_rent_csv = pd.read_csv("brooklyn_rent_remaining_zipcodes.csv")

brooklyn_rent_csv

,zipcode,year,borough,mean_rent,count
0,11201,2015,BROOKLYN,3227.864,NaN
1,11201,2016,BROOKLYN,3271.231,NaN
2,11201,2017,BROOKLYN,3292.957,NaN
3,11201,2018,BROOKLYN,3352.501,NaN
4,11201,2019,BROOKLYN,3488.163,NaN
...,...,...,...,...,...
226,11249,2018,BROOKLYN,3566.972,NaN
227,11249,2019,BROOKLYN,3718.571,NaN
228,11249,2020,BROOKLYN,3812.068,NaN
229,11249,2021,BROOKLYN,3999.540,NaN


In [32]:
manhattan_rent = non_intersecting[non_intersecting["borough"] == "MANHATTAN"]
display(manhattan_rent)

,zipcode,year,borough,mean_rent,count
0,10001,2015,MANHATTAN,4010.002,NaN
1,10001,2016,MANHATTAN,4022.986,NaN
2,10001,2017,MANHATTAN,3958.379,NaN
3,10001,2018,MANHATTAN,4001.064,NaN
4,10001,2019,MANHATTAN,4145.263,NaN
...,...,...,...,...,...
331,10282,2018,MANHATTAN,5750.776,NaN
332,10282,2019,MANHATTAN,6012.092,NaN
333,10282,2020,MANHATTAN,5772.033,NaN
334,10282,2021,MANHATTAN,5967.184,NaN


In [33]:
manhattan_rent.to_csv("manhattan_rent_remaining_zipcodes.csv", index=False)

In [34]:
manhattan_rent_csv = pd.read_csv("manhattan_rent_remaining_zipcodes.csv")

manhattan_rent_csv

,zipcode,year,borough,mean_rent,count
0,10001,2015,MANHATTAN,4010.002,NaN
1,10001,2016,MANHATTAN,4022.986,NaN
2,10001,2017,MANHATTAN,3958.379,NaN
3,10001,2018,MANHATTAN,4001.064,NaN
4,10001,2019,MANHATTAN,4145.263,NaN
...,...,...,...,...,...
326,10282,2018,MANHATTAN,5750.776,NaN
327,10282,2019,MANHATTAN,6012.092,NaN
328,10282,2020,MANHATTAN,5772.033,NaN
329,10282,2021,MANHATTAN,5967.184,NaN


In [38]:
staten_island_rent = non_intersecting[non_intersecting["borough"] == "STATEN_ISLAND"]
display(staten_island_rent)

,zipcode,year,borough,mean_rent,count
336,10301,2022,STATEN_ISLAND,2290.967,NaN
337,10304,2020,STATEN_ISLAND,2397.822,NaN
338,10304,2021,STATEN_ISLAND,2116.676,NaN
339,10304,2022,STATEN_ISLAND,2100.812,NaN
340,10305,2021,STATEN_ISLAND,1893.651,NaN
341,10305,2022,STATEN_ISLAND,2038.290,NaN
342,10306,2021,STATEN_ISLAND,1660.265,NaN
343,10306,2022,STATEN_ISLAND,1897.448,NaN
344,10308,2022,STATEN_ISLAND,2432.923,NaN
345,10309,2022,STATEN_ISLAND,1925.669,NaN


In [39]:
staten_island_rent.to_csv("staten_island_rent_remaining_zipcodes.csv", index=False)

In [40]:
staten_island_rent = pd.read_csv("staten_island_rent_remaining_zipcodes.csv")

staten_island_rent

,zipcode,year,borough,mean_rent,count
0,10301,2022,STATEN_ISLAND,2290.967,NaN
1,10304,2020,STATEN_ISLAND,2397.822,NaN
2,10304,2021,STATEN_ISLAND,2116.676,NaN
3,10304,2022,STATEN_ISLAND,2100.812,NaN
4,10305,2021,STATEN_ISLAND,1893.651,NaN
5,10305,2022,STATEN_ISLAND,2038.290,NaN
6,10306,2021,STATEN_ISLAND,1660.265,NaN
7,10306,2022,STATEN_ISLAND,1897.448,NaN
8,10308,2022,STATEN_ISLAND,2432.923,NaN
9,10309,2022,STATEN_ISLAND,1925.669,NaN
